## Solution for HW - 3: Classification

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('data/course_lead_scoring.csv')
df.head(3)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1


In [3]:
df.head(3).T

,0,1,2
lead_source,paid_ads,social_media,events
industry,NaN,retail,healthcare
number_of_courses_viewed,1,1,5
annual_income,79450.0,46992.0,78796.0
employment_status,unemployed,employed,unemployed
location,south_america,south_america,australia
interaction_count,4,1,3
lead_score,0.94,0.8,0.69
converted,1,0,1


### 🧹 Data Preparation

Check if the missing values are presented in the features.  

If there are missing values:  
- For **categorical features**, replace them with `'NA'`  
- For **numerical features**, replace them with `0.0`

In [4]:
df.isnull().sum().sort_values(ascending=False)

annual_income               181
industry                    134
lead_source                 128
employment_status           100
location                     63
number_of_courses_viewed      0
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
categorical_col = list(df.dtypes[df.dtypes == 'object'].index)
display(categorical_col)

numeric_col = list(df.dtypes[(df.dtypes == 'int64') | (df.dtypes == 'float64')].index)
display(numeric_col)

['lead_source', 'industry', 'employment_status', 'location']

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [6]:
df[categorical_col] = df[categorical_col].fillna('NA')
df[numeric_col] = df[numeric_col].fillna(0)

In [7]:
# Check if we have any missing number - No
df.isnull().sum().sort_values(ascending=False)

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

 - 1.  What is the most frequent observation (mode) for the column industry?

In [8]:
df.industry.mode()

0    retail
Name: industry, dtype: object

- 2. Create the **correlation matrix** for the numerical features of your dataset.  
In a correlation matrix, you compute the **correlation coefficient** between every pair of numerical features.

Now, identify **which two features** among the following have the **highest correlation**:

- `interaction_count` and `lead_score`  
- `number_of_courses_viewed` and `lead_score`  
- `number_of_courses_viewed` and `interaction_count`  
- `annual_income` and `interaction_count`  

In [9]:
# Correlation matrix
corr = df.corr(numeric_only=True).round(3)

# Define the pairs to compare
pairs = {
    ('interaction_count', 'lead_score'): corr.loc['interaction_count', 'lead_score'],
    ('number_of_courses_viewed', 'lead_score'): corr.loc['number_of_courses_viewed', 'lead_score'],
    ('number_of_courses_viewed', 'interaction_count'): corr.loc['number_of_courses_viewed', 'interaction_count'],
    ('annual_income', 'interaction_count'): corr.loc['annual_income', 'interaction_count']
}

# Find the pair with the highest correlation - pairs.get is a function that retrieves the value for a given key
# pairs.get(('interaction_count', 'lead_score')) = 0.84. max() iterates through the keys of pairs. For each key, it calls pairs.get(key) → returns the value (the correlation).
max_pair = max(pairs, key=pairs.get) # Find the key in the dictionary whose value (correlation coefficient) is the largest
max_value = pairs[max_pair]

print(f"The pair with the highest correlation is {max_pair} with correlation = {max_value:.3f}")

The pair with the highest correlation is ('annual_income', 'interaction_count') with correlation = 0.027


Split your dataset into **train**, **validation**, and **test** sets with the following proportions:

- **Training set:** 60%  
- **Validation set:** 20%  
- **Test set:** 20%  

Use **Scikit-Learn’s** `train_test_split` function and set the random seed to `42` for reproducibility.  

Make sure that the **target variable** (the one you’re trying to predict) is **not included** in your feature dataframe.

In [10]:
df_full_train, df_test = train_test_split(df,
                        test_size=0.2,
                        random_state=1
                        )
df_train, df_val = train_test_split(df_full_train,
                        test_size=0.25,
                        random_state=1)

In [11]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [12]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [13]:
# .values ensures that we take the values of the churn variable. Otherwise, we will take an index and the values.
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

# Delete the outcome variable so that we don't use it as a control variable
X_train = df_train.drop('converted', axis = 1)
X_val = df_val.drop('converted', axis = 1)
X_test = df_test.drop('converted', axis = 1)

- 3. Calculate the **mutual information score** between the target variable `converted` and the other **categorical variables** in the dataset.  
Use **only the training set** for this calculation.

Round each score to **2 decimal places** using `round(score, 2)`.

Identify which of the following variables has the **highest mutual information score**:

- `industry`  
- `location`  
- `lead_source`  
- `employment_status`  


In [14]:
def mutual_info_score_func(series):
    return metrics.mutual_info_score(series, df_full_train.converted).round(4)

In [15]:
mi = df_full_train[categorical_col].apply(mutual_info_score_func)
mi.sort_values(ascending=False)

lead_source          0.0246
employment_status    0.0127
industry             0.0082
location             0.0012
dtype: float64

- 4. Now let's train a **logistic regression**.    

Remember that we have several **categorical variables** in the dataset. Include them using **one-hot encoding**.  
Fit the model on the **training dataset**.  
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:  

    ```python
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    ```

Calculate the **accuracy** on the **validation dataset** and round it to **2 decimal digits**.

What accuracy did you get?

- `0.64`  
- `0.74`  
- `0.84`  
- `0.94`


- Remove our dependent variable `converted` from the dataset

In [16]:
# handle_unknown='error'). If you don’t specify this argument, the default is 'error'. That means if your validation or test set contains a category 
# that was not present in the training data, you’ll get an error like this: ValueError: Found unknown categories ['telecom'] in column 0 during transform
ohe = OneHotEncoder(handle_unknown='ignore')
# fit_transform() = Learns how to encode (finds all unique categories, builds mapping) and applies the encoding. Used only on the training set
X_train_enc = ohe.fit_transform(X_train)
# transform() = Uses the same encoding rules learned from training data and applies them to new data. Used on validation/test sets
X_val_enc = ohe.transform(X_val)

In [17]:
model = linear_model.LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
# solver='lbfgs' is the default solver in newer version of sklearn
# for older versions, you need to specify it explicitly
model.fit(X_train_enc, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [18]:
y_proba_pred = model.predict_proba(X_val_enc)[:, 1]
y_pred = (y_proba_pred >= 0.5).astype(int)
y_pred

array([0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1])

In [19]:
print(f'Accurace score is {round(metrics.accuracy_score(y_val, y_pred), 3)}')

Accurace score is 0.829


Goal: Find the least useful feature using the feature elimination technique.

Steps:
- 1️⃣ Train a model using the same features and parameters as in Question 4 (without rounding).
- 2️⃣ Exclude each feature one by one and train a new model without it.
- 3️⃣ Record the accuracy for each model.
- 4️⃣ Calculate the difference between the original accuracy and the accuracy without each feature.
- 5️⃣ The feature with the smallest difference is the least useful.

In [20]:
# Fit encoder only once on full training data
ohe = OneHotEncoder(handle_unknown='ignore')
X_train_enc = ohe.fit_transform(X_train)
X_val_enc = ohe.transform(X_val)

# Train the full model
model = linear_model.LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_enc, y_train)
full_acc = metrics.accuracy_score(y_val, model.predict(X_val_enc))

features = ['industry', 'employment_status', 'lead_score']
diffs = {}

for f in features:
    # Drop feature *before encoding*, but transform using the same ohe categories
    X_train_new = X_train.drop(f, axis=1)
    X_val_new = X_val.drop(f, axis=1)
    
    # Important: use *the same fitted encoder* here. fill_value='NA' ensures that deleted categorical column is filled with NA
    X_train_new_enc = ohe.transform(X_train_new.reindex(columns=X_train.columns, fill_value='NA'))
    X_val_new_enc = ohe.transform(X_val_new.reindex(columns=X_train.columns, fill_value='NA'))
    
    temp_model = linear_model.LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    temp_model.fit(X_train_new_enc, y_train)
    acc = metrics.accuracy_score(y_val, temp_model.predict(X_val_new_enc))
    diffs[f] = full_acc - acc

print(diffs)
print("Least important feature:", min(diffs, key=lambda k: abs(diffs[k])))


{'industry': -0.0068259385665528916, 'employment_status': 0.017064846416382284, 'lead_score': 0.010238907849829393}
Least important feature: industry


Now let's train a regularized logistic regression.

- Let's try the following values of the parameter **C**: `[0.01, 0.1, 1, 10, 100]`.
- Train models using all the features as in **Question 4**.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

**Question:**  
Which of these **C** values leads to the best accuracy on the validation set?

- 0.01  
- 0.1  
- 1  
- 10  
- 100  

> **Note:** If there are multiple options, select the smallest **C**.

In [32]:
# Fit encoder only once on full training data
ohe = OneHotEncoder(handle_unknown='ignore')
X_train_enc = ohe.fit_transform(X_train)
X_val_enc = ohe.transform(X_val)

C = [0.01, 0.1, 1, 10, 100]
results = {}

for param in C:
    # Train the full model
    model = linear_model.LogisticRegression(solver='liblinear', 
                                            C=param, 
                                            penalty='l1',# type of regularization
                                            max_iter=1000, 
                                            random_state=42)
    model.fit(X_train_enc, y_train)
    full_acc = metrics.accuracy_score(y_val, model.predict(X_val_enc))
    results[param] = full_acc

In [33]:
results

{0.01: 0.5836177474402731,
 0.1: 0.7440273037542662,
 1: 0.8327645051194539,
 10: 0.825938566552901,
 100: 0.8020477815699659}

In [35]:
best_C = max(results, key=results.get) # Find the key in the dictionary whose value (correlation coefficient) is the largest
best_acc = results[best_C]

print(f"The pair with the highest correlation is with C equal to {best_C} with correlation = {best_acc:.3f}")

The pair with the highest correlation is with C equal to 1 with correlation = 0.833
